# Breast Cancer Detection
## Experiments
This notebook, unlike bcd_experiment, runs a single experiment at a time.

## Python Packages

In [1]:
!pip install --upgrade pip --quiet
!pip install wandb --upgrade --quiet
!pip install python-dotenv --quiet
!pip install keras==2.15.0 --quiet
!pip install keras-core --quiet
!pip install tensorflow --quiet

In [2]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb
import numpy as np
import logging

from base_network import NetworkConfig
# Model Architectures
from base_network import Network
from aknet import AKNetConfig, AKNetFactory
from mlnet import MLNetConfig, MLNetFactory
from nlnet import NLNetConfig, NLNetFactory
from nlnetv2 import NLNetV2Config, NLNetV2Factory
# from nlnetv3 import NLNetV3Config, NLNetV3Factory
from nlnetv4 import NLNetV4Config, NLNetV4Factory
from nlnetv6 import NLNetV6Config, NLNetV6Factory
from simplenetv2 import SimpleNetV2Config, SimpleNetV2Factory
from tmnet import TMNetConfig, TMNetFactory
from tmnetv3 import TMNetV3Config, TMNetV3Factory
from tmnetv4 import TMNetV4Config, TMNetV4Factory
from tmnetv6 import TMNetV6Config, TMNetV6Factory
from tmnetv7 import TMNetV7Config, TMNetV7Factory
from zznetv1 import ZZNetV1Config, ZZNetV1Factory
from zznetv1 import ZZNetV1Config, ZZNetV1Factory
from zznetv2 import ZZNetV2Config, ZZNetV2Factory
from zznetv3 import ZZNetV3Config, ZZNetV3Factory

# Repository controlling persistence of models and experiments
from store import ExperimentRepo

# Base Models
from pretrained import DenseNet, MobileNet, Xception, Inception, VGG

# Experiment driver
from experiment import FeatureExtractionExperiment

# Configuration Objects
from config import ProjectConfig, DatasetConfig, CheckPointConfig, TrainConfig, EarlyStopConfig, LearningRateScheduleConfig, Config, ExperimentConfig

# Adapter controls access to secrets, dataset directories, and distribute strategies.
from adapter import KaggleAdapter, Adapter

## Parameters

In [3]:
# MAKE SURE DESCRIPTION ACCURATELY REFLECTS THIS RUN.
mode = "Production"
force=False
model_id =["TMNetV7_DenseNet_16",
"NLNetV2_DenseNet_71",
"NLNetV4_DenseNet_72",
"NLNetV6_MobileNet_73",
"ZZNetV2_MobileNet_74"]

network_name = ["tmnetv7",
"nlnetv2",
"nlnetv4",
"nlnetv6",
"zznetv2"]


base_model = [    
DenseNet(),
DenseNet(),
DenseNet(),
MobileNet(),
MobileNet()]


## Adapter 
The adapter object encapsulates variables that are platform-dependent, such as device type, distribute strategy, api keys, file paths, etc...

In [4]:
adapter = KaggleAdapter(mode=mode)

# Obtain the TensorFlow state and compute distribution policy, i.e. strategy
strategy  = adapter.get_strategy()

# Weights and Biases login for model and metric tracking.
wandb.login(key=adapter.wandb_api_key)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Initializing the TPU system: local


2024-04-10 02:00:25.906792: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.906907: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.907004: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.907106: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.907173: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.907373: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:25.907460: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:***

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Reproducibility

In [5]:
def seed_everything():
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(hash("improves reproducibility") % 2**32 - 1)
    tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)
seed_everything()

## Network Configurations and Factories

In [6]:
def get_network(name: str)-> Network:   
    
    if "convnet" == name:
        config = ConvNetConfig(dense=1024)
        factory = ConvNetFactory(config=config)
        network = {"config": config, "factory": factory}
        return network  
        
    if "aknet" == name:
        config = AKNetConfig(dense1=4096,
                              dropout1=0.5,
                              dense2=2048,
                              dropout2=0.5)
        factory = AKNetFactory(config=config)
        network = {"config": config, "factory": factory}
        return network  
        
    if "mlnet" == name:
        config = MLNetConfig(dropout0=0.5,
                              dense1=1024,
                              dropout1=0.5,
                              dense2=512,
                              dropout2=0.5)
        factory = MLNetFactory(config=config)
        network = {"config": config, "factory": factory}
        return network    
    
    if "nlnet" == name:
        config = NLNetConfig(dense1=4096,
                             dense2=4096)
        factory = NLNetFactory(config=config)
        network = {"config": config, "factory": factory}
        return network        
    
    if "nlnetv2" == name:
        config = NLNetV2Config(dense1=2048,
                               dense2=2048,
                               dense3=1024)
        factory = NLNetV2Factory(config=config)
        network = {"config": config, "factory": factory}
        return network
    
    if "nlnetv3" == name:
        config = NLNetV3Config(dense1=4096,
                               dense2=4096,
                               dense3=2048,
                               l2_1=0.01,
                              l2_2=0.01,
                              l2_3=0.01)
        factory = NLNetV3Factory(config=config)
        network = {"config": config, "factory": factory}
        return network    
        
    if "nlnetv4" == name:
        config = NLNetV4Config(dense1=2048,
                               dense2=2048,
                               dense3=1024)
        factory = NLNetV4Factory(config=config)
        network = {"config": config, "factory": factory}
        return network        
    
    if "nlnetv5" == name:
        config = NLNetV5Config(dense1=4096,
                               dropout1=0.8,
                               dense2=4096,
                               dropout2=0.8,
                               dense3=2048,
                              dropout3=0.8)
        factory = NLNetV5Factory(config=config)
        network = {"config": config, "factory": factory}
        return network
    
    if "nlnetv6" == name:
        config = NLNetV6Config(dense1=1024,
                               dense2=1024,
                               dense3=1024)
        factory = NLNetV6Factory(config=config)
        network = {"config": config, "factory": factory}
        return network   
    
    if "simplenetv2" == name:
        config = SimpleNetV2Config(dropout1=0.5,
                                dense1=256)
        factory = SimpleNetV2Factory(config=config)
        network = {"config": config, "factory": factory}
        return network       

    if "tmnet" == name:
        config = TMNetConfig(dense1=1024,
                               dense2=1024)
        factory = TMNetFactory(config=config)
        network = {"config": config, "factory": factory}
        return network
    
    if "tmnetv3" == name:
        config = TMNetV3Config(dense1=4096,
                               dense2=4096,
                              dense3=1024)
        factory = TMNetV3Factory(config=config)
        network = {"config": config, "factory": factory}
        return network    
        
    if "tmnetv4" == name:
        config = TMNetV4Config(dense1=4096,
                               l21=0.01,
                               dropout1=0.5,
                               dense2=4096,
                               l22=0.01,
                               dropout2=0.5,
                               dense3=1024,
                               l23=0.01,
                               dropout3=0.5,
                               dense4=1024,
                               l24=0.01,
                               dropout4=0.5)
        factory = TMNetV4Factory(config=config)
        network = {"config": config, "factory": factory}
        return network  
    

    if "tmnetv6" == name:
        config = TMNetV6Config(dense1=4096,
                               dropout1=0.5,
                               dense2=4096,
                               dropout2=0.5,
                               dense3=2048,
                               dropout3=0.5,
                               dense4=2048,
                               dropout4=0.5)
        factory = TMNetV6Factory(config=config)
        network = {"config": config, "factory": factory}
        return network

    if "tmnetv7" == name:
        config = TMNetV7Config(dense1=1024,
                               l2reg1=0.0001,
                               dense2=1024,
                               l2reg2=0.0001)
        factory = TMNetV7Factory(config=config)
        network = {"config": config, "factory": factory}
        return network
    
    if "zznetv1" == name:
        config = ZZNetV1Config(dense1=2048,
                               dense2=1024,
                               dense3=1024,
                               dropout3=0.5)
        factory = ZZNetV1Factory(config=config)
        network = {"config": config, "factory": factory}
        return network    
    
    if "zznetv2" == name:
        config = ZZNetV2Config(dense1=2048,
                               dense2=1024,
                               dropout2=0.5)
        factory = ZZNetV2Factory(config=config)
        network = {"config": config, "factory": factory}
        return network       
    
    if "zznetv3" == name:
        config = ZZNetV3Config(dense1=1024,                               
                               dropout1=0.6)
        factory = ZZNetV3Factory(config=config)
        network = {"config": config, "factory": factory}
        return network           

## Build Experiment Configuration

In [7]:
def build_config(adapter: Adapter, mode: str,network_config: Config, strategy: tf.distribute.Strategy) -> Config:
    """Constructs an experiment Config object """
    # Encapsulates the parameters that define the project in Weights & Biases
    project_config = ProjectConfig(mode=mode)

    # The default batch size is 64; however, if running on TPU, the rule of thumb is to optimally set the batch size to 128 * the number of TPU cores (8).     
    batch_size = 64 if not adapter.device_type == "TPU" else 8 * strategy.num_replicas_in_sync    
    dataset_config = DatasetConfig(mode=mode,
                                   seed=123,
                                   batch_size=batch_size)

    # If running on TPU, the learning rate is scaled by the number of cores.
    learning_rate = 1e-3 if not adapter.device_type == "TPU" else 1e-4 * strategy.num_replicas_in_sync
    train_config = TrainConfig(epochs=100, 
                               learning_rate=learning_rate,
                               optimizer="RMSprop",
                               use_ema=False,
                               momentum=0.0,
                               weight_decay=None,
                               loss="binary_crossentropy",
                               early_stop=True,
                               learning_rate_schedule=True,
                               augmentation=True,
                               enhance=False,
                               checkpoint=False,
                               fine_tune=False,
    )    

    # Checkpoints will be stored in the directory given by the adapter object. 
    checkpoint_config = CheckPointConfig(directory=adapter.model_dir, 
                                         monitor="val_loss", 
                                         verbose=1, 
                                         save_best_only=True, 
                                         save_weights_only=True, 
                                         mode="auto")

    # We'll establish an early stop callback to mitigate overfitting caused by excessive training after validation loss hasn't improved.
    early_stop_config = EarlyStopConfig(min_delta=1e-4, 
                                        monitor="val_loss", 
                                        patience=50, 
                                        restore_best_weights=True, 
                                        verbose=1)
    
    # Learning rate configuration contains the default values for the schedule. 
    learning_rate_schedule_config = LearningRateScheduleConfig(method="reduce_on_plateau",
                                                              min_delta=1e-4,
                                                              min_lr=1e-5,
                                                              monitor="val_loss",
                                                              factor=0.5,
                                                              patience=3,
                                                              restore_best_weights=True,
                                                              verbose=1,
                                                              mode="auto")

    # The experiment configuration is encapsulated into a single object 
    return ExperimentConfig(project=project_config, 
                            dataset=dataset_config, 
                            train=train_config, 
                            network=network_config, 
                            checkpoint=checkpoint_config, 
                            early_stop=early_stop_config,
                            learning_rate_schedule=learning_rate_schedule_config                           
                           )

## Build Dataset

In [8]:

def build_dataset(train_dir: str, subset: str, dataset_config: Config) -> tf.data.Dataset:
    """Produces a TensorFlow training or validation  Dataset  """
    train_dir = pathlib.Path(train_dir).with_suffix('') 
    return tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels=dataset_config.labels,
        color_mode=dataset_config.color_mode,
        image_size=dataset_config.image_size,
        shuffle=dataset_config.shuffle,
        validation_split=dataset_config.validation_split,
        subset=subset,
        interpolation=dataset_config.interpolation,
        seed=dataset_config.seed,
        batch_size=dataset_config.batch_size
    )

## Dataset Augmentation

In [9]:
def optimize_dataset(train_ds: tf.data.Dataset, augment: bool = True, enhance: bool = False) -> tf.data.Dataset:
    
    hflip = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal')
    ])
    
    vflip = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('vertical')
    ])
        
    rotate = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(0.2)
    ])
   
    if enhance:
    
        # Horizontal flip the original dataset.
        train_ds2 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (hflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE)) 
        
        # Vertically flip the original dataset.
        train_ds3 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (vflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))         

        # Rotate the original dataset
        train_ds4 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))            

        # Rotate the horizontally flipped dataset
        train_ds5 = (train_ds2
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))     
        
        # Rotate the vertically flipped dataset
        train_ds6 = (train_ds3
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))             

        # Concatenate the datasets
        train_ds = train_ds.concatenate(train_ds2)
        train_ds = train_ds.concatenate(train_ds3)
        train_ds = train_ds.concatenate(train_ds4)
        train_ds = train_ds.concatenate(train_ds5)
        train_ds = train_ds.concatenate(train_ds6)
    
    elif augment:
        # Horizontal flip the original dataset.
        train_ds = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (hflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE)) 
        
        # Vertically flip the original dataset.
        train_ds = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (vflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))         

        # Rotate the original dataset
        train_ds = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))       
    
    else:
        train_ds = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .prefetch(tf.data.AUTOTUNE))               
        
        
    return train_ds

## Build Callbacks
The build_callbacks function creates callbacks common to all networks. Early stop is one such callback that is optionally created based upon the train config.

The add_learning_rate_callback adds a network-specific learning rate schedule. In this case, the Triangle learning rate policy is created for each network, based upon the minimum and maximum learning rates obtained by the learning rate range test of each network.

In [10]:
def build_callbacks(config: Config) -> list:
    """Construct an early stop, learning rate, and model checkpoint callback. """    
    
    callbacks = []
    
    if config.train.early_stop:
    
        early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                            min_delta=config.early_stop.min_delta,
                                                            patience=config.early_stop.patience, 
                                                            restore_best_weights=config.early_stop.restore_best_weights,
                                                            verbose=config.early_stop.verbose)
        callbacks.append(early_stop_callback)
        
    if config.train.learning_rate_schedule:
        
        if config.learning_rate_schedule.method == "reduce_on_plateau":

            lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                                    factor=config.learning_rate_schedule.factor,
                                                                    patience=config.learning_rate_schedule.patience,
                                                                    verbose=config.learning_rate_schedule.verbose,
                                                                    mode=config.learning_rate_schedule.mode,
                                                                    min_delta=config.learning_rate_schedule.min_delta,
                                                                    min_lr=config.learning_rate_schedule.min_lr)
            callbacks.append(lr_callback)

        if config.learning_rate_schedule.method == "triangle":

            lr_callback = TriangleLearningRateScheduleCallback(name=name,
                                                               min_lr=config.learning_rate_schedule.min_lr,
                                                               max_lr=config.learning_rate_schedule.max_lr,
                                                               step_size=config.learning_rate_schedule.stepsize,
                                                              )

            callbacks.append(lr_callback)
    return callbacks

## Optimizer

Currently supporting Adam and RMSprop optimizers.

In [11]:
def get_optimizer(config: Config):
    if config.train.optimizer == "Adam":
        return tf.keras.optimizers.Adam(learning_rate=config.train.learning_rate,
                                use_ema=config.train.use_ema,
                                ema_momentum=config.train.momentum,
                                weight_decay=config.train.weight_decay)         
    elif config.train.optimizer == "RMSprop":
        return tf.keras.optimizers.RMSprop(learning_rate=config.train.learning_rate,
                                            use_ema=config.train.use_ema,
                                            ema_momentum=config.train.momentum,
                                            weight_decay=config.train.weight_decay)    
    else:
        raise ValueError("Invalid optimizer")

## Build and Run Experiments

For each pretrained base model, the following steps are executed:

1. Instantiate a TensorFlow strategy scope context.
2. Create a network object based on the specified base/pretrained model. 
3. Add a network-specific learning rate schedule to the callbacks.
4. Designate tags used for search purposes on Weights & Biases
3. Construct an experiment object for the network
4. Run the experiment.

In [ ]:
for i in range(len(network_name)):
    # Get the network. We evaluate one network at a time.
    network = get_network(name=network_name[i])

    # Build configuration object
    config = build_config(adapter=adapter,mode=mode, network_config=network["config"], strategy=strategy)    

    # Extract W&B note from network config.
    notes = f"{config.network.description}"

    # Repository controls persistence of experiment, runs, and models
    repo = ExperimentRepo(mode=mode, project=config.project.name, adapter=adapter)

    # Early stop and learning rate callbacks 
    callbacks = build_callbacks(config=config)

    # Construct datasets and optimize and preprocess training set
    train_ds = build_dataset(train_dir=adapter.train_dir, subset="training", dataset_config=config.dataset)
    train_ds = optimize_dataset(train_ds=train_ds, augment=config.train.augmentation, enhance=config.train.enhance)    
    val_ds = build_dataset(train_dir=adapter.train_dir, subset="validation", dataset_config=config.dataset)

    # Extract the factory object from the network dictionary
    factory = network["factory"]

    with strategy.scope(): 
        metrics = ['accuracy', tf.keras.metrics.AUC()]
        # Instantiate the optimizer with the current configuration    
        optimizer = get_optimizer(config=config)           
        # Instantiate a network including the designated base model.
        network = factory.create(base_model=base_model[i])
        # Tags allowing models and runs to be searched on Weights and Biases
        tags = [model_id[i], network.name, network.architecture, "augmented_dataset"]        
        # The Experiment object compiles and fits the model.
        experiment = FeatureExtractionExperiment(
            network=network, 
            config=config, 
            optimizer=optimizer, 
            repo=repo, 
            metrics=metrics, 
            callbacks=callbacks, tags=tags, notes=notes, force=force)
        experiment.run(train_ds=train_ds, val_ds=val_ds)

2024-04-10 02:00:32.366215: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366265: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366291: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366375: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366450: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366636: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 02:00:32.366722: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

Found 2471 files belonging to 2 classes.
Using 1977 files for training.
Found 2471 files belonging to 2 classes.
Using 494 files for validation.


I0000 00:00:1712714434.157094      13 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74836368/74836368 [==============================] - 0s 0us/step
# ================================================================================================ #
                                          TMNetV7_DenseNet                                          
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 TMNetV7_DenseNet_input_lay  [(None, 224, 224, 3)]     0         
 er (InputLayer)                                                 
                                                                 
 tf.math.truediv (TFOpLambd  (None, 224, 224, 3)       0         
 a)                                                              
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         
 )                     

wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin


...

epoch/accuracy,▁▂▃▃▄▄▅▆▆▇▇▇▇▇▇█▇█▇█████████▇▇█▇████████
epoch/auc,▁▂▃▄▄▅▆▆▇▇▇█████████████████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,█████▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,██████▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▅▅▆▆██▄▆▇█▇▇▇▇▇██▇▇▇█▇▇█████▇▇▇▇▇▇█▇▇▇▇
epoch/val_auc,▁▃▄▄▄▆▇▅▇▆▇██▇▇▇▇█▆▇█▇▆▇▇▇▇▇▇▇▇▇▇▆▆▇▇▇▇▇
epoch/val_loss,▂▃▁▁▁▂▄▅▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇████████
epoch/accuracy,0.90339
epoch/auc,0.96533


Found 2471 files belonging to 2 classes.
Using 1977 files for training.
Found 2471 files belonging to 2 classes.
Using 494 files for validation.
# ================================================================================================ #
                                          NLNetV2_DenseNet                                          
# ------------------------------------------------------------------------------------------------ #
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NLNetV2_DenseNet_input_lay  [(None, 224, 224, 3)]     0         
 er (InputLayer)                                                 
                                                                 
 tf.math.truediv_2 (TFOpLam  (None, 224, 224, 3)       0         
 bda)                                                            
                                                                 
 tf.nn.

Epoch 1/100


2024-04-10 02:14:43.626058: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


31/31 [==============================] - 116s 2s/step - loss: 2.0058 - accuracy: 0.5458 - auc_1: 0.5626 - val_loss: 0.7116 - val_accuracy: 0.5668 - val_auc_1: 0.5579 - lr: 8.0000e-04
Epoch 2/100
31/31 [==============================] - 8s 248ms/step - loss: 1.2712 - accuracy: 0.5402 - auc_1: 0.5597 - val_loss: 1.8690 - val_accuracy: 0.4474 - val_auc_1: 0.5545 - lr: 8.0000e-04
Epoch 3/100
31/31 [==============================] - 8s 248ms/step - loss: 0.9735 - accuracy: 0.5847 - auc_1: 0.6003 - val_loss: 0.8313 - val_accuracy: 0.5506 - val_auc_1: 0.5473 - lr: 8.0000e-04
Epoch 4/100
31/31 [==============================] - ETA: 0s - loss: 0.8525 - accuracy: 0.5928 - auc_1: 0.6242
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00039999998989515007.
31/31 [==============================] - 8s 247ms/step - loss: 0.8525 - accuracy: 0.5928 - auc_1: 0.6242 - val_loss: 0.8109 - val_accuracy: 0.5729 - val_auc_1: 0.5717 - lr: 8.0000e-04
Epoch 5/100
31/31 [==============================] - 